In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import json

In [2]:
df = pd.read_csv('nicknames.csv')

In [3]:
df.dropna(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141248 entries, 1 to 141250
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   nick_name  141248 non-null  object
dtypes: object(1)
memory usage: 2.2+ MB


In [5]:
df_p = pd.read_csv('profanity_en.csv')

In [6]:
df_p

,text,canonical_form_1,canonical_form_2,canonical_form_3,category_1,category_2,category_3,severity_rating,severity_description
0,69,69,NaN,NaN,sexual anatomy / sexual acts,NaN,NaN,1.0,Mild
1,@55,ass,NaN,NaN,sexual anatomy / sexual acts,NaN,NaN,1.0,Mild
2,@ssfcker,fuck,ass,NaN,sexual anatomy / sexual acts,sexual orientation / gender,NaN,2.8,Severe
3,@ssfucker,fuck,ass,NaN,sexual anatomy / sexual acts,sexual orientation / gender,NaN,2.8,Severe
4,@ssfvcker,fuck,ass,NaN,sexual anatomy / sexual acts,sexual orientation / gender,NaN,2.4,Strong
...,...,...,...,...,...,...,...,...,...
1593,wnker,wank,NaN,NaN,sexual anatomy / sexual acts,other / general insult,NaN,1.0,Mild
1594,wop,wop,NaN,NaN,racial / ethnic slurs,NaN,NaN,2.4,Strong
1595,wophead,wop,NaN,NaN,racial / ethnic slurs,NaN,NaN,2.2,Strong
1596,zip in the wire,zipperhead,NaN,NaN,racial / ethnic slurs,NaN,NaN,1.4,Mild


In [7]:
profanity = df_p['text'].tolist()

In [8]:
nicknames = df['nick_name'].tolist()

In [9]:
df_sw_en = pd.read_csv('bad-words.csv')

df_sw_en = df_sw_en.rename(columns={"jigaboo":"word"})

In [10]:
df_sw = pd.read_json("data.json")

In [11]:
df_sw_concat = pd.concat([df_sw_en, df_sw, pd.DataFrame({'word':profanity})], ignore_index=True)

In [12]:
df_sw_concat

,word
0,mound of venus
1,asslover
2,s&m
3,queaf
4,whitetrash
...,...
3510,wnker
3511,wop
3512,wophead
3513,zip in the wire


In [13]:
df_sw_concat.to_csv('swear_words.csv', index=None)

In [18]:
df_sw_concat['is_offensive'] = 1
df_sw_concat.rename(columns={'word':'text'}, inplace=True)

In [19]:
df_sw_concat.to_csv('swear_words.csv', index=None)

In [20]:
data = df_sw_concat

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(data.word).toarray()
labels = data.is_offensive
features.shape

(3515, 65)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(data['word'], data['is_offensive'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [37]:
print(clf.predict_proba(count_vect.transform(["hello"]))[0])

[1.]


In [29]:
def predict(word):
    return clf.predict(count_vect.transform([word]))[0]

In [32]:
df['is_offensive'] = df['nick_name'].apply(predict)

In [34]:
df[df['is_offensive']==0]

,nick_name,is_offensive


In [31]:
swear_words = df_sw_concat['word'].tolist()

In [32]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

sentences = swear_words
embeddings = model.encode(sentences)
embeds = []

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    embeds.append(embedding)
    
df_sw_concat['embed'] = embeds

In [33]:
df_sw_concat

,word,embed
0,mound of venus,"[-0.0012805157, 0.00071827986, -0.003063048, 0..."
1,asslover,"[0.013014284, 0.00821337, -0.030748818, -0.029..."
2,s&m,"[0.039454542, 0.01372775, -0.00965603, 0.00609..."
3,queaf,"[0.027138958, 0.025423177, -0.03308883, -0.017..."
4,whitetrash,"[-0.0055718645, 0.037569765, -0.027341058, -0...."
...,...,...
3510,wnker,"[0.008764469, 0.0034810754, -0.018183053, -0.0..."
3511,wop,"[0.043651517, 0.022549834, -0.021462087, -0.05..."
3512,wophead,"[0.023418043, 0.024824606, -0.0039961813, -0.0..."
3513,zip in the wire,"[-0.018007044, -0.011607245, -0.07618794, 0.00..."


In [34]:
sentences = nicknames
embeddings = model.encode(sentences)
embeds = []

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    embeds.append(embedding)
    
df['embed'] = embeds

In [35]:
df

,nick_name,embed
0,NaN,"[0.011652807, 0.009943119, -0.02741476, -0.035..."
1,‌ ‌ ‌ ‌ ‌ ‌ ‌ ‌?,"[0.02485765, -0.0080936635, -0.04724872, -0.07..."
2,‌ ‌ ‌- emilia<3,"[0.033218604, -0.011780523, -0.035756074, 0.00..."
3,⁭⁫⁪⁫⁬⁭⁫⁪⁫⁬ ⁭hop_pa,"[0.050225995, 0.03244206, -0.016792454, 0.0230..."
4,‌ (Z),"[0.0041125217, 0.05452588, -0.05655993, -0.016..."
...,...,...
141246,𝓜𝓸𝓱𝓪𝓶𝓶𝓪𝓭 𝓡𝓮𝔃𝓪,"[0.009509911, -0.008516736, -0.015367321, -0.0..."
141247,🤍🖤🤎💜💙💚💛🧡⁦❤️⁩,"[0.01359888, -0.003466455, -0.027659642, -0.03..."
141248,𝐌𝐚𝐫𝐳𝐢𝐲𝐞𝐡🌱,"[0.01359888, -0.003466455, -0.027659642, -0.03..."
141249,𝓐𝓶𝓲𝓻𝓱𝓸𝓼𝓮𝓲𝓷,"[0.01359888, -0.003466455, -0.027659642, -0.03..."


In [36]:
swears = []
for i in range(len(df)):
    score = cosine_similarity(df_sw_concat['embed'].tolist(), df['embed'].iloc[i].reshape(1, -1))
    argmax = np.argmax(score)
    if score[argmax][0] > 0.98:
        swears.append(df['nick_name'].iloc[i])

In [37]:
len(swears)

188

In [38]:
for s in swears:
    print(s)

69
a_s_h_sh
a_s_r_a
abo
allah
amateur
American
anal
arab
areola
ass
asss
babe
barf
bbc
bibi
black
caca
canadian
cool
crash
cristian
Death
Demon
desire
devil
devill
dick
die
doodoo
Dumb
ehsj
ero
fack
faith
fat
fatah
fatahne
fear
fhg
fight
fire
fuck
fuck you
fucker
gangbang
gay
gin
god
goddamnrad
gun
guts
hab
hajj
hama
hamse
harem
hell
hiv
ho
homo
idiot
Italiano
jaf
japoni
jiso
juya
kid
kill
kin
kink
kinko
kkk
kkkk
klan
knife
Latin
lolita
loser
lucifer
mafia
mama
mamaj
mamas
mami
mamia
mamina
mammadi
mamo
mams
mfh
mfk
milf
moslem
murder
muslim
muslimgh
negah
ney
nia
niaish
nicka
nono
noon
osama
panty
pee
peter
po
pom
poo
poon
pooo
poya
randy
roach
russian
s_a
s_u_d_eh
s.b.h
s&m
sambo
satan
sex
shit
sima
slut
sniper
sob
sos
swallow
tit
uk
usama
vfm
wahaj
wf
Whiskey
willie
willy
wtf
xx
xxx
xxxx
ya
yaa
zo
zoa
اسب
انگل
انی
بدبخت
پدرت
پفیوز
ترک
جنده
خر
خری
خوب
زارت
سولاخ
سیاه
شوتی
عرب
عربي
عربية
عمه ننه
عن
فارس
کثافت
کرم
کس
کس‌کش
کص
کص خل
کون
کونی
کیر
گاو
گوز
گوه
لاشی
لر
ملنگ
نگایدم
نگاییدم
ن

In [43]:
def check_swear(word):
    word = str(word)
    for i in range(len(df_sw_concat)):
        swear = str(df_sw_concat.iloc[i]['word'])
        if word in swear or swear in word:
            return 1

In [44]:
df['is_swear'] = df['nick_name'].apply(check_swear)

KeyboardInterrupt: 

In [ ]:
pd.DataFrame({'swear':swears})

In [ ]:
score = cosine_similarity(df['embed'].tolist(), df_sw['embed'].iloc[4].reshape(1, -1))

In [ ]:
argmax = np.argmax(score)

In [ ]:
score[argmax][0]

In [ ]:
df.iloc[argmax]['nick_name']

In [39]:
'sex' in 'hamedsexy'

True